<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Project Setup
* This notebok describes how to create and configure a project
* This is the same as creating a new project in the editor and going through all of the steps.

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * A Batch must be created

In [ ]:
!pip install "labelbox[data]" -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [ ]:
import labelbox as lb
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY=""
client = lb.Client(api_key=API_KEY)

### Create Dataset
* Create dataset and attach data
* More details on attaching data can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/data_rows.ipynb)

In [ ]:
dataset = client.create_dataset(name="project_setup_demo")
global_keys = ["id1", "id2", "id3", "id4"]
## Example image
uploads = []
# Generate data rows
for i in range(1,5):
    uploads.append({
        'row_data':  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": global_keys[i-1]
    })
task = dataset.create_data_rows(uploads)
task.wait_till_done()
print("ERRORS: " , task.errors)
print("RESULT URL: ", task.result_url)

ERRORS:  None
RESULT URL:  https://storage.googleapis.com/lb-labelbox-193903-adv/metadata/orgs/cl3ahv73w1891087qbwzs3edd/tasks/c88xpab2t0729066ylm6fvs5t/datarow-import-result.json?GoogleAccessId=sa-lb-api-0da28c-7505d6%40labelbox-193903.iam.gserviceaccount.com&Expires=1695135734&Signature=WAOWrKsTEemO88dz%2BGvttmZ9WHiBRFSatcqyA9OCMml8xFnK2DEnaDDlO5HFAZdRQrRK18FYvUpRjP%2BSneNolOvxRE57iuAeR04WKRt%2FCxSGRG%2B4x9Jc0KhubXw7i%2BS2GaeK9%2FLK5Tf3rmpUSib00hTdEsOCMw87gus9iBKP1g7m4Iw7jZVGfJm9tLLtOEjhgENCYvF44ITQ%2BB1qBgKz4vLsL7Gi5%2BP%2BiW%2BXaQABB2MZ2Np7z9wVFXqRJx82b3TrJVXTHDEtY9WiBi%2Fp8LxXAc7QOQruUaFOpEoD97PpigwoQpJL6ZaVry%2FLcZtP%2BEEjB5ETVXKfVr%2FGLaUNPXG6fA%3D%3D&response-content-disposition=attachment


### Create Or Select an Ontology
* Optionally create an ontology or select from an existing one.
* More details on ontology management can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/ontologies.ipynb)
   

In [ ]:
# Create
ontology_builder = lb.OntologyBuilder(
    tools=[lb.Tool(name="cat", tool=lb.Tool.Type.BBOX)])
ontology = client.create_ontology("project-setup-demo-ontology",
                                  ontology_builder.asdict())
# Select existing ontology
#ontology = client.get_ontology("<ontology_id>")
#ontology = existing_project.ontology()

### Create Project and Setup the Editor
* Setting up a project will add an ontology and will enable labeling to begin
* Creating batches will add all data_rows belonging to the dataset to the queue.

In [ ]:

batch_project = client.create_project(name="Project Setup Demo",
                                quality_mode=lb.QualityMode.Consensus,
                                media_type=lb.MediaType.Image)


batch_project.setup_editor(ontology)


AttributeError: module 'labelbox' has no attribute 'QualityMode'

# Add data to your projects 

In [ ]:

## When creating a batch you can also setup the data rows priority
batch = batch_project.create_batch(
  "batch-demo-4-2", # Each batch in a project must have a unique name
  global_keys=global_keys, # A list of data rows or data row ids
  priority=5, # priority between 1(Highest) - 5(lowest)
  consensus_settings={
    "numberOfLabels": 2, 
    "coveragePercentage": 0.1 
  } 
)
print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-09-05 14:59:51+00:00",
    "name": "batch-demo-4-2",
    "size": 4,
    "uid": "ddb40b70-4bfc-11ee-8b96-936d5f85b4f1",
    "updated_at": "2023-09-05 14:59:51+00:00"
}>


In [ ]:
batch_project.batches().get_one().consensus_settings_json



'{"numberOfLabels":1,"coveragePercentage":0}'

### Review

In [ ]:
# Note setup_complete will be None if it fails.
print(batch_project.setup_complete)
print(batch_project.ontology())
print([ds.name for ds in batch_project.batches()])
print(batch_project.)


2023-09-05 14:55:22+00:00
<Ontology {
    "classification_schema_count": 0,
    "created_at": "2023-09-05 14:55:17.551000+00:00",
    "description": null,
    "name": "project-setup-demo-ontology",
    "normalized": {
        "id": "clm6fmwpr0up907xu0lmc5cle",
        "name": "project-setup-demo-ontology",
        "tools": [
            {
                "schemaNodeId": "clm6fmwqg0upb07xu0m5g9wxl",
                "featureSchemaId": "clm6fmwqg0upa07xu2xwn3gus",
                "required": false,
                "name": "cat",
                "tool": "rectangle",
                "color": "#ff0000",
                "archived": 0,
                "classifications": [],
                "kind": "Rectangle"
            }
        ],
        "classifications": [],
        "relationships": []
    },
    "object_schema_count": 1,
    "uid": "clm6fmwpr0up907xu0lmc5cle",
    "updated_at": "2023-09-05 14:55:17.551000+00:00"
}>
['batch-demo-4']


In [ ]:
print(f"https://app.labelbox.com/projects/{batch_project.uid}")

https://app.labelbox.com/projects/clm6fmzr20cfb07yf095f4sf5


# Cleanup

In [ ]:
# batch_project.delete()
# dataset_project.delete()
# dataset.delete()